In [1]:
import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: karpovaknn (use `wandb login --relogin` to force relogin)


True

In [3]:
import sys
sys.path.insert(0, "./mmdetection")

import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
from mmdet.apis import set_random_seed

# Imports
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import random
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

1.6.0+cu101 True
10.1
GCC 7.3


In [4]:
global_seed = 111

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

## Prepare config

In [5]:
from mmcv import Config

baseline_cfg_path = "../mmdetection/configs/cascade_rcnn/cascade_rcnn_x101_32x4d_fpn_1x_coco.py"
cfg = Config.fromfile(baseline_cfg_path)

In [6]:
model_name = 'cascade_rcnn_x101_32x4d_fpn_1x_train_887_map0_5_0_95_aug_PMDistortion_final'
fold = 0
job = 4

# Folder to store model logs and weight files
job_folder = f'job{job}_{model_name}_fold{fold}'
cfg.work_dir = job_folder

# Change the wnd username and project name below
wnb_username = 'karpovaknn'
wnb_project_name = 'defect_detection'

# Set seed thus the results are more reproducible
cfg.seed = global_seed

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: job4_cascade_rcnn_x101_32x4d_fpn_1x_train_887_map0_5_0_95_aug_PMDistortion_final_fold0


In [7]:
# Set the number of classes
for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

cfg.gpu_ids = [0]

# Setting pretrained model in the init_cfg which is required 
# for transfer learning as per the latest MMdetection update
cfg.model.backbone.init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d')
cfg.model.pop('pretrained', None)

cfg.runner.max_epochs = 30 # Epochs for the runner that runs the workflow 
cfg.total_epochs = 30

# Learning rate of optimizers. The LR is divided by 8 since the config file is originally for 8 GPUs
cfg.optimizer.lr = 0.02/8

## Learning rate scheduler config used to register LrUpdater hook
cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

# config to register logger hook
cfg.log_config.interval = 20 # Interval to print the log

# Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
cfg.checkpoint_config.interval = 1 # The save interval is 1

In [8]:
cfg.dataset_type = 'CocoDataset' # Dataset type, this will be used to define the dataset
cfg.classes = ("scratch",)

cfg.data.train.img_prefix = 'dataset/train' # Prefix of image path
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = f'dataset/train/coco.json'
cfg.data.train.type='CocoDataset'

cfg.data.val.img_prefix = 'dataset/val' # Prefix of image path
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = f'dataset/val/coco.json'
cfg.data.val.type='CocoDataset'

cfg.data.test.img_prefix = 'dataset/test' # Prefix of image path
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file =  f'dataset/test/coco.json'
cfg.data.test.type='CocoDataset'

cfg.data.samples_per_gpu = 2 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 2 # Worker to pre-fetch data for each single GPU

In [9]:
# The config to build the evaluation hook, refer to https://github.com/open-mmlab/mmdetection/blob/master/mmdet/core/evaluation/eval_hooks.py#L7 for more details.
cfg.evaluation.metric = 'bbox' # Metrics used during evaluation

# Set the epoch intervel to perform evaluation
cfg.evaluation.interval = 1

# Set the iou threshold of the mAP calculation during evaluation
# cfg.evaluation.iou_thrs = [0.5]
# cfg.evaluation.iou_thrs = [0.5, 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95]

In [17]:
# albu_train_transforms = [
#     dict(type='ShiftScaleRotate', shift_limit=0.0625,
#          scale_limit=0.15, rotate_limit=15, p=0.4),
#     dict(type='RandomBrightnessContrast', brightness_limit=0.2,
#          contrast_limit=0.2, p=0.5),
#     dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.4),
#     dict(type="Blur", p=1.0, blur_limit=7),
#     dict(type='CLAHE', p=0.5),
#     dict(type='Equalize', mode='cv', p=0.4),
#     dict(
#         type="OneOf",
#         transforms=[
#             dict(type="GaussianBlur", p=1.0, blur_limit=7),
#             dict(type="MedianBlur", p=1.0, blur_limit=7),
#         ],
#         p=0.4,
#     ),
#     ]

albu_train_transforms = [
    dict(type='PhotoMetricDistortion', brightness_delta=32,
         contrast_range=(0.5, 1.5), saturation_range=(0.5, 1.5),
         hue_delta=18),
    dict(type='ShiftScaleRotate', shift_limit=0.0625,
         scale_limit=0.15, rotate_limit=15, p=0.4)
#     dict(type='RandomFlip', flip_ratio = 0.5),
#     dict(type='RandomRotate90'),
#     dict(type='Flip'),
#     dict(type='RandomCrop')
#     dict(type='PhotoMetricDistortion'),
#     dict(type='ColorTransform')
    ]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True
    ),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [19]:
cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project=wnb_project_name,
                                              name=f'exp-{model_name}-fold{fold}-job{job}',
                                              entity=wnb_username))
                       ]

In [20]:
cfg_path = f'{job_folder}/job{job}_{Path(baseline_cfg_path).name}'
print(cfg_path)

# Save config file for inference later
cfg.dump(cfg_path)
print(f'Config:\n{cfg.pretty_text}')

job4_cascade_rcnn_x101_32x4d_fpn_1x_train_887_map0_5_0_95_aug_PMDistortion_final_fold0/job4_cascade_rcnn_x101_32x4d_fpn_1x_coco.py
Config:
model = dict(
    type='CascadeRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d'),
        groups=32,
        base_width=4),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='Delt

## Build a model and train

In [21]:
model = build_detector(cfg.model,
                       train_cfg=cfg.get('train_cfg'),
                       test_cfg=cfg.get('test_cfg'))
model.init_weights()

2022-02-22 09:49:52,446 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_32x4d'}
2022-02-22 09:49:52,448 - mmcv - INFO - load model from: open-mmlab://resnext101_32x4d
2022-02-22 09:49:52,448 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_32x4d
2022-02-22 09:49:52,856 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-02-22 09:49:52,888 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-02-22 09:49:52,898 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-02-22 09:49:52,993 - mmcv - INFO - initialize Shared2FCBBo

2022-02-22 09:49:53,180 - mmcv - INFO - 
backbone.layer2.0.downsample.1.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,180 - mmcv - INFO - 
backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,181 - mmcv - INFO - 
backbone.layer2.1.conv1.weight - torch.Size([256, 512, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,181 - mmcv - INFO - 
backbone.layer2.1.bn1.weight - torch.Size([256]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,181 - mmcv - INFO - 
backbone.layer2.1.bn1.bias - torch.Size([256]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,181 - mmcv - INFO - 
backbone.layer2.1.conv2.weight - torch.Size([256, 8, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,182 - mmcv - INFO - 
backbone.layer2

2022-02-22 09:49:53,199 - mmcv - INFO - 
backbone.layer3.2.conv2.weight - torch.Size([512, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,199 - mmcv - INFO - 
backbone.layer3.2.bn2.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,199 - mmcv - INFO - 
backbone.layer3.2.bn2.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,200 - mmcv - INFO - 
backbone.layer3.2.conv3.weight - torch.Size([1024, 512, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,200 - mmcv - INFO - 
backbone.layer3.2.bn3.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,200 - mmcv - INFO - 
backbone.layer3.2.bn3.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,200 - mmcv - INFO - 
backbone.layer3.3.conv1.weigh

2022-02-22 09:49:53,213 - mmcv - INFO - 
backbone.layer3.8.bn1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,214 - mmcv - INFO - 
backbone.layer3.8.conv2.weight - torch.Size([512, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,214 - mmcv - INFO - 
backbone.layer3.8.bn2.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,214 - mmcv - INFO - 
backbone.layer3.8.bn2.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,215 - mmcv - INFO - 
backbone.layer3.8.conv3.weight - torch.Size([1024, 512, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,215 - mmcv - INFO - 
backbone.layer3.8.bn3.weight - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,215 - mmcv - INFO - 
backbone.layer3.8.bn3.bias - t

2022-02-22 09:49:53,234 - mmcv - INFO - 
backbone.layer3.14.bn1.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,234 - mmcv - INFO - 
backbone.layer3.14.bn1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,234 - mmcv - INFO - 
backbone.layer3.14.conv2.weight - torch.Size([512, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,235 - mmcv - INFO - 
backbone.layer3.14.bn2.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,235 - mmcv - INFO - 
backbone.layer3.14.bn2.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,235 - mmcv - INFO - 
backbone.layer3.14.conv3.weight - torch.Size([1024, 512, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,235 - mmcv - INFO - 
backbone.layer3.14.bn3.we

2022-02-22 09:49:53,248 - mmcv - INFO - 
backbone.layer3.20.conv1.weight - torch.Size([512, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,248 - mmcv - INFO - 
backbone.layer3.20.bn1.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,249 - mmcv - INFO - 
backbone.layer3.20.bn1.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,249 - mmcv - INFO - 
backbone.layer3.20.conv2.weight - torch.Size([512, 16, 3, 3]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,249 - mmcv - INFO - 
backbone.layer3.20.bn2.weight - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,250 - mmcv - INFO - 
backbone.layer3.20.bn2.bias - torch.Size([512]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,250 - mmcv - INFO - 
backbone.layer3.20.conv3.

2022-02-22 09:49:53,271 - mmcv - INFO - 
backbone.layer4.2.bn2.bias - torch.Size([1024]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,272 - mmcv - INFO - 
backbone.layer4.2.conv3.weight - torch.Size([2048, 1024, 1, 1]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,272 - mmcv - INFO - 
backbone.layer4.2.bn3.weight - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,272 - mmcv - INFO - 
backbone.layer4.2.bn3.bias - torch.Size([2048]): 
PretrainedInit: load from open-mmlab://resnext101_32x4d 
 
2022-02-22 09:49:53,273 - mmcv - INFO - 
neck.lateral_convs.0.conv.weight - torch.Size([256, 256, 1, 1]): 
XavierInit: gain=1, distribution=uniform, bias=0 
 
2022-02-22 09:49:53,273 - mmcv - INFO - 
neck.lateral_convs.0.conv.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of CascadeRCNN  
 
2022-02-22 09:49:53,273 - mmcv - INFO - 
neck.la

In [22]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [23]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-02-22 09:50:09,086 - mmdet - INFO - Start running, host: jovyan@defect-detection-0, work_dir: /home/jovyan/defect_detection/job4_cascade_rcnn_x101_32x4d_fpn_1x_train_887_map0_5_0_95_aug_PMDistortion_final_fold0
2022-02-22 09:50:09,087 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdate

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


2022-02-22 09:50:11.327310: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2022-02-22 09:50:32,478 - mmdet - INFO - Epoch [1][20/248]	lr: 9.740e-05, eta: 1:41:55, time: 0.824, data_time: 0.137, memory: 6405, loss_rpn_cls: 0.6757, loss_rpn_bbox: 0.0269, s0.loss_cls: 0.3727, s0.acc: 91.9092, s0.loss_bbox: 0.0017, s1.loss_cls: 0.2452, s1.acc: 78.5400, s1.loss_bbox: 0.0006, s2.loss_cls: 0.1065, s2.acc: 95.9375, s2.loss_bbox: 0.0001, loss: 1.4295
2022-02-22 09:50:45,180 - mmdet - INFO - Epoch [1][40/248]	lr: 1.973e-04, eta: 1:29:59, time: 0.635, data_time: 0.005, memory: 6405, loss_rpn_cls: 0.4491, loss_rpn_bbox: 0.0175, s0.loss_cls: 0.0480, s0.acc: 99.4482, s0.loss_bbox: 0.0030, s1.loss_cls: 0.0204, s1.acc: 99.5166, s1.loss_bbox: 0.0009, s2.loss_cls: 0.0144, s2.acc: 99.5459, s2.loss_bbox: 0.0001, loss: 0.5532
2022-02-22 09:50:57,858 - mmdet - INFO - Epoch [1][60/248]	lr: 2.972e-04, eta: 1:25:49, time: 0.634, data_time: 0.005, memory: 6405, loss_rpn_cls: 0.0934, loss_rpn_bbox: 0.0262, s0.loss_cls: 0.1107, s0.acc: 98.7451, s0.loss_bbox: 0.0194, s1.loss_cls: 0.0334,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 167/167, 8.1 task/s, elapsed: 21s, ETA:     0s

...

2022-02-22 11:21:12,332 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2022-02-22 11:21:12,401 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.648
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.225
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.479
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.410
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).


learning_rate,▂▄▆█████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▇▄▅▅▅▅▆▆█▅▆▅▆▅▅▅▅▄▄▃▄▄▄▄▂▂▃▂▂▁▁▁▂▁▂▁▁▁▁▂
train/loss_rpn_bbox,█▆▅▆▅▆▆▄▅▃▃▃▄▃▃▃▃▃▂▂▂▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_cls,█▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.acc,▄▄▃▁▂▃▂▃▁▃▄▄▅▆▄▅▆▆▅▇▆▆▆▆▇▇▇▇▇██████▇▇▇▇▇
train/s0.loss_bbox,▅▄▆▇▆▅▆▇█▆▅▅▅▅▅▄▄▃▃▃▄▃▄▄▂▂▃▂▂▁▁▁▂▁▂▁▂▁▁▂
train/s0.loss_cls,█▆▇▆▅▅▅▅▆▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▂▂
train/s1.acc,▇█▆▅▂▃▁▁▁▂▂▄▃▅▄▅▅▆▃▆▅▆▆▅▇▇▇▇▇███▇███▇██▇
train/s1.loss_bbox,▁▁▂▃▄▄▅▆█▆▇▆▇▆▆▆▆▅▅▄▆▅▆▅▃▃▃▃▂▂▂▂▂▂▃▂▂▂▂▂
train/s1.loss_cls,█▄▅▆▆▆▇▇█▆▆▅▅▄▅▄▄▃▅▃▄▃▃▃▂▂▂▂▁▁▁▁▂▁▁▁▂▁▁▂


In [24]:
# Get the best epoch number
import json
from collections import defaultdict

log_file = f'{job_folder}/None.log.json'

# Source: mmdetection/tools/analysis_tools/analyze_logs.py 
def load_json_logs(json_logs):
    # load and convert json_logs to log_dict, key is epoch, value is a sub dict
    # keys of sub dict is different metrics, e.g. memory, bbox_mAP
    # value of sub dict is a list of corresponding values of all iterations
    log_dicts = [dict() for _ in json_logs]
    for json_log, log_dict in zip(json_logs, log_dicts):
        with open(json_log, 'r') as log_file:
            for line in log_file:
                log = json.loads(line.strip())
                # skip lines without `epoch` field
                if 'epoch' not in log:
                    continue
                epoch = log.pop('epoch')
                if epoch not in log_dict:
                    log_dict[epoch] = defaultdict(list)
                for k, v in log.items():
                    log_dict[epoch][k].append(v)
    return log_dicts

log_dict = load_json_logs([log_file])
# [(print(inner['bbox_mAP']) for inner in item) for item in log_dict]
# [print(item) for item in log_dict[0]]
best_epoch = np.argmax([item['bbox_mAP'][0] for item in log_dict[0].values()])+1
best_epoch

16

In [25]:
model_files = [f'{job_folder}/epoch_{best_epoch}.pth',
               cfg_path
              ]

# Create a new wnb run for saving models as artifacts
run = wandb.init(project=wnb_project_name,
                 name=f'models_files_{model_name}_fold{fold}_job{job}',
                 entity=wnb_username,
                 group='Artifact',
                 job_type='model-files')

artifact = wandb.Artifact(f'models_files_{model_name}_fold{fold}_job{job}', type='model')

for model_file in model_files:
    artifact.add_file(model_file)

run.log_artifact(artifact)
run.finish()

2022-02-22 11:24:20.371448: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
